<a href="https://colab.research.google.com/github/ADAMFUTUR/ADAMFUTUR/blob/main/cactus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AnasBerka/CactiViT-materials.git

Cloning into 'CactiViT-materials'...
remote: Enumerating objects: 5932, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 5932 (delta 14), reused 14 (delta 5), pack-reused 5887 (from 1)
Receiving objects: 100% (5932/5932), 735.39 MiB | 24.24 MiB/s, done.
Resolving deltas: 100% (48/48), done.
Updating files: 100% (6037/6037), done.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from transformers import ViTForImageClassification, ViTFeatureExtractor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from PIL import Image
import warnings
warnings.filterwarnings("ignore")



In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:

# Data preprocessing for ViT
vit_transform = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
cnn_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load CIFAR-10 dataset (replace with your dataset, e.g., cactus images)
train_dataset_vit = datasets.CIFAR10(root="./data", train=True, download=True,
                                     transform=lambda x: vit_transform(Image.fromarray(np.array(x)))["pixel_values"][0])
test_dataset_vit = datasets.CIFAR10(root="./data", train=False, download=True,
                                    transform=lambda x: vit_transform(Image.fromarray(np.array(x)))["pixel_values"][0])
train_dataset_cnn = datasets.CIFAR10(root="./data", train=True, download=True, transform=cnn_transform)
test_dataset_cnn = datasets.CIFAR10(root="./data", train=False, download=True, transform=cnn_transform)

train_loader_vit = DataLoader(train_dataset_vit, batch_size=32, shuffle=True)
test_loader_vit = DataLoader(test_dataset_vit, batch_size=32, shuffle=False)
train_loader_cnn = DataLoader(train_dataset_cnn, batch_size=32, shuffle=True)
test_loader_cnn = DataLoader(test_dataset_cnn, batch_size=32, shuffle=False)

# Vision Transformer (ViT)
vit_model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224", num_labels=10, ignore_mismatched_sizes=True)
vit_model = vit_model.to(device)
vit_optimizer = optim.Adam(vit_model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# ResNet (Deep Learning Model)
resnet_model = models.resnet18(pretrained=True)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 10)
resnet_model = resnet_model.to(device)
resnet_optimizer = optim.Adam(resnet_model.parameters(), lr=1e-3)

# EfficientNet (Deep Learning Model)
efficientnet_model = models.efficientnet_b0(pretrained=True)
efficientnet_model.classifier[1] = nn.Linear(efficientnet_model.classifier[1].in_features, 10)
efficientnet_model = efficientnet_model.to(device)
efficientnet_optimizer = optim.Adam(efficientnet_model.parameters(), lr=1e-3)

# Training function for deep learning models
def train_model(model, train_loader, optimizer, criterion, epochs=3):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs) if isinstance(model, models.ResNet) or isinstance(model, models.EfficientNet) else model(inputs).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

# Evaluation function for deep learning models
def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs) if isinstance(model, models.ResNet) or isinstance(model, models.EfficientNet) else model(inputs).logits
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Train and evaluate ViT
print("Training ViT...")
train_model(vit_model, train_loader_vit, vit_optimizer, criterion)
vit_accuracy = evaluate_model(vit_model, test_loader_vit)
print(f"ViT Accuracy: {vit_accuracy:.4f}")

# Train and evaluate ResNet
print("Training ResNet...")
train_model(resnet_model, train_loader_cnn, resnet_optimizer, criterion)
resnet_accuracy = evaluate_model(resnet_model, test_loader_cnn)
print(f"ResNet Accuracy: {resnet_accuracy:.4f}")

# Train and evaluate EfficientNet
print("Training EfficientNet...")
train_model(efficientnet_model, train_loader_cnn, efficientnet_optimizer, criterion)
efficientnet_accuracy = evaluate_model(efficientnet_model, test_loader_cnn)
print(f"EfficientNet Accuracy: {efficientnet_accuracy:.4f}")

# Prepare data for traditional ML models (flatten images)
def prepare_ml_data(dataset):
    X, y = [], []
    for img, label in dataset:
        X.append(np.array(img).flatten())
        y.append(label)
    return np.array(X), np.array(y)

X_train_ml, y_train_ml = prepare_ml_data(train_dataset_cnn)
X_test_ml, y_test_ml = prepare_ml_data(test_dataset_cnn)

# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_ml, y_train_ml)
rf_accuracy = accuracy_score(y_test_ml, rf_model.predict(X_test_ml))
print(f"Random Forest Accuracy: {rf_accuracy:.4f}")

# SVM
svm_model = SVC(kernel="rbf", random_state=42)
svm_model.fit(X_train_ml[:5000], y_train_ml[:5000])  # Subset for speed
svm_accuracy = accuracy_score(y_test_ml, svm_model.predict(X_test_ml))
print(f"SVM Accuracy: {svm_accuracy:.4f}")